In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import pprint

import folium
from folium.plugins import MarkerCluster
import googlemaps
import geopandas as gpd

import requests
import json
import time

import warnings
warnings.filterwarnings(action='ignore')

In [19]:
bus_time_table = pd.read_csv('./datasets/bus_time_table.csv', encoding='cp949', sep=',', parse_dates=['기준_날짜'])

In [20]:
state_geo = gpd.read_file('./datasets/seoul_geo_dong.geojson')
state_geo.head(5)

,OBJECTID,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8,geometry
0,1,서울특별시 종로구 사직동,1101053,1111053000,11110,11,서울특별시,종로구 사직동,종로구,11010530,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,2,서울특별시 종로구 삼청동,1101054,1111054000,11110,11,서울특별시,종로구 삼청동,종로구,11010540,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,3,서울특별시 종로구 부암동,1101055,1111055000,11110,11,서울특별시,종로구 부암동,종로구,11010550,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,4,서울특별시 종로구 평창동,1101056,1111056000,11110,11,서울특별시,종로구 평창동,종로구,11010560,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,5,서울특별시 종로구 무악동,1101057,1111057000,11110,11,서울특별시,종로구 무악동,종로구,11010570,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."


In [22]:
junggu_geo = state_geo[ state_geo['adm_cd'].astype('int')//100 == 11020 ]
junggu_geo

,OBJECTID,adm_nm,adm_cd,adm_cd2,sgg,sido,sidonm,temp,sggnm,adm_cd8,geometry
17,18,서울특별시 중구 소공동,1102052,1114052000,11140,11,서울특별시,중구 소공동,중구,11020520,"MULTIPOLYGON (((126.98109 37.56180, 126.98050 ..."
18,19,서울특별시 중구 회현동,1102054,1114054000,11140,11,서울특별시,중구 회현동,중구,11020540,"MULTIPOLYGON (((126.98536 37.55424, 126.98543 ..."
19,20,서울특별시 중구 명동,1102055,1114055000,11140,11,서울특별시,중구 명동,중구,11020550,"MULTIPOLYGON (((126.97703 37.56919, 126.97758 ..."
20,21,서울특별시 중구 필동,1102057,1114057000,11140,11,서울특별시,중구 필동,중구,11020570,"MULTIPOLYGON (((126.99347 37.56300, 126.99339 ..."
21,22,서울특별시 중구 장충동,1102058,1114058000,11140,11,서울특별시,중구 장충동,중구,11020580,"MULTIPOLYGON (((127.00957 37.56303, 127.00936 ..."
22,23,서울특별시 중구 광희동,1102059,1114059000,11140,11,서울특별시,중구 광희동,중구,11020590,"MULTIPOLYGON (((127.00957 37.56303, 127.00670 ..."
23,24,서울특별시 중구 을지로동,1102060,1114060500,11140,11,서울특별시,중구 을지로동,중구,11020600,"MULTIPOLYGON (((126.99347 37.56300, 126.99143 ..."
24,25,서울특별시 중구 신당5동,1102065,1114065000,11140,11,서울특별시,중구 신당5동,중구,11020650,"MULTIPOLYGON (((127.02362 37.56517, 127.02671 ..."
25,26,서울특별시 중구 황학동,1102067,1114067000,11140,11,서울특별시,중구 황학동,중구,11020670,"MULTIPOLYGON (((127.02362 37.56517, 127.02059 ..."
26,27,서울특별시 중구 중림동,1102068,1114068000,11140,11,서울특별시,중구 중림동,중구,11020680,"MULTIPOLYGON (((126.97140 37.55960, 126.97057 ..."


In [6]:
my_loc

,id,dong_name,dong_code,dong_code_8,geometry
17,18,서울특별시 중구 소공동,1102052,11140520,POINT (126.97492 37.56475)
18,19,서울특별시 중구 회현동,1102054,11140540,POINT (126.97960 37.55636)
19,20,서울특별시 중구 명동,1102055,11140550,POINT (126.98444 37.56216)
20,21,서울특별시 중구 필동,1102057,11140570,POINT (126.99416 37.55767)
21,22,서울특별시 중구 장충동,1102058,11140580,POINT (127.00171 37.55503)
22,23,서울특별시 중구 광희동,1102059,11140590,POINT (127.00271 37.56480)
23,24,서울특별시 중구 을지로동,1102060,11140605,POINT (126.99660 37.56621)
24,25,서울특별시 중구 신당동,1102069,11140615,POINT (127.01375 37.56763)
25,26,서울특별시 중구 다산동,1102070,11140625,POINT (127.00667 37.55183)
26,27,서울특별시 중구 약수동,1102071,11140635,POINT (127.01063 37.55076)


In [16]:
junggu_geo

,id,dong_name,dong_code,dong_code_8,geometry
17,18,서울특별시 중구 소공동,1102052,11140520,POINT (126.97492 37.56475)
18,19,서울특별시 중구 회현동,1102054,11140540,POINT (126.97960 37.55636)
19,20,서울특별시 중구 명동,1102055,11140550,POINT (126.98444 37.56216)
20,21,서울특별시 중구 필동,1102057,11140570,POINT (126.99416 37.55767)
21,22,서울특별시 중구 장충동,1102058,11140580,POINT (127.00171 37.55503)
22,23,서울특별시 중구 광희동,1102059,11140590,POINT (127.00271 37.56480)
23,24,서울특별시 중구 을지로동,1102060,11140605,POINT (126.99660 37.56621)
24,25,서울특별시 중구 신당동,1102069,11140615,POINT (127.01375 37.56763)
25,26,서울특별시 중구 다산동,1102070,11140625,POINT (127.00667 37.55183)
26,27,서울특별시 중구 약수동,1102071,11140635,POINT (127.01063 37.55076)


In [25]:
with open('./datasets/seoul_geo_sigugun.json',mode='rt',encoding='utf-8') as f:
    geo = json.loads(f.read())
    f.close()
    
my_loc =  geo['features'][-2]

here = [37.560914, 126.990202]
m = folium.Map(location=here, tiles="OpenStreetMap", zoom_start=14)

folium.GeoJson(junggu_geo,name='seoul_municipalities').add_to(m)
m